In [1]:
import time
import os
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from kafka import KafkaProducer

In [2]:
#Đẩy lên kafka
producer = KafkaProducer(bootstrap_servers="localhost:9092")

with open("VCB Historical Data.csv", "r") as f:
    for line in f:
        producer.send("n2_ict", line.encode())

producer.flush()

In [ ]:

class FileChangeHandler(FileSystemEventHandler):
    def __init__(self, file_path, kafka_topic, producer):
        self.file_path = file_path
        self.kafka_topic = kafka_topic
        self.producer = producer
        self.last_position = os.path.getsize(file_path)
    
    def on_modified(self, event):
        if event.src_path == self.file_path:
            with open(self.file_path, 'r') as file:
                file.seek(self.last_position)
                new_lines = file.readlines()
                self.last_position = file.tell()
                
                for line in new_lines:
                    self.producer.send(self.kafka_topic, line.encode('utf-8'))
                    print(f"Sent to Kafka: {line.strip()}")

def watch_file(file_path, kafka_topic):
    producer = KafkaProducer(bootstrap_servers='localhost:9092')
    event_handler = FileChangeHandler(file_path, kafka_topic, producer)
    
    # Kiểm tra và in ra đường dẫn
    directory = os.path.dirname(file_path)
    print(f"Watching directory: {directory}")
    
    observer = Observer()
    observer.schedule(event_handler, path=directory, recursive=False)
    observer.start()
    
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

# Sử dụng đường dẫn tuyệt đối
file_path = r"D:\SIC-BD1\Result\VCB Historical Data.csv"
watch_file(file_path, 'n2_ict')


Watching directory: D:\SIC-BD1\Result
